In [1]:
import sys
!"{sys.executable}" -m pip install pyspark findspark


[notice] A new release of pip is available: 23.2.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataWarehouseAnalysis").getOrCreate()

**Loading Data**

In [ ]:
from pathlib import Path
import os

# Portable: repo_root/data  (works on any machine)
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
DATA_DIR = Path(os.getenv("DATA_DIR", str(PROJECT_ROOT / "data")))

if not DATA_DIR.exists():
    raise FileNotFoundError(
        f"DATA_DIR not found: {DATA_DIR}\n"
        f"Put CSV files in: {PROJECT_ROOT/'data'}\n"
        f"Or set an environment variable DATA_DIR to your dataset folder."
    )

# List all CSV files in the data folder
csv_files = [p for p in DATA_DIR.iterdir() if p.suffix.lower() == ".csv"]

dataframes = {}
for p in csv_files:
    df_name = p.stem  # filename without .csv
    dataframes[df_name] = spark.read.csv(str(p), header=True, inferSchema=True)

print("Loaded:", list(dataframes.keys()))



Loaded: ['DimAccount', 'DimCurrency', 'DimCustomer', 'DimDate', 'DimDepartmentGroup', 'DimGeography', 'DimOrganization', 'DimProduct', 'DimProductCategory', 'DimProductSubcategory', 'DimPromotion', 'DimReseller', 'DimSalesReason', 'DimSalesTerritory', 'DimScenario', 'FactCallCenter', 'FactCurrencyRate', 'FactFinance', 'FactInternetSales', 'FactSalesTargets']


**Data Cleaning** 

In [4]:
from pyspark.sql.functions import col, count, when

missing_values = {}
for df_name, df in dataframes.items():
    # Count missing values for each column
    missing_count = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).collect()
    missing_values[df_name] = {col: val for col, val in zip(df.columns, missing_count[0])}

# The dictionary 'missing_values' will have the count of missing values for each column in each DataFrame



**Missing Values in each DataFrame**

In [5]:
from pyspark.sql.functions import col, count, when

missing_values = {}
for df_name, df in dataframes.items():
    # Count missing values for each column
    missing_count = df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).collect()
    missing_values[df_name] = {col: val for col, val in zip(df.columns, missing_count[0])}

# The dictionary 'missing_values' will have the count of missing values for each column in each DataFrame



**Dropping Unnecessary Columns from Customer Table**

In [6]:

# Dropping 'MiddleName', 'AddressLine2' and 'Title' columns from 'DimCustomer'
dimCustomer = dataframes['DimCustomer']
dimCustomer = dimCustomer.drop('MiddleName', 'AddressLine2', 'Title', 'Suffix')

# Update the DataFrame dictionary
dataframes['DimCustomer'] = dimCustomer


**Handling Outliers** 

In [7]:
from pyspark.sql.functions import lit, percentile_approx, col

dimCustomer = dataframes['DimCustomer']

# List of numerical columns to check for outliers
numerical_columns = ['YearlyIncome', 'TotalChildren', 'NumberChildrenAtHome'] 

outliers = {}
for col_name in numerical_columns:
    # Calculate the lower and upper quartiles
    bounds = dimCustomer.stat.approxQuantile(col_name, [0.25, 0.75], 0.05)
    IQR = bounds[1] - bounds[0]

    # Define lower and upper bounds for outliers
    lower_bound = bounds[0] - 1.5 * IQR
    upper_bound = bounds[1] + 1.5 * IQR

    # Count outliers
    outliers_below = dimCustomer.filter(col(col_name) < lit(lower_bound)).count()
    outliers_above = dimCustomer.filter(col(col_name) > lit(upper_bound)).count()

    # Store the count of outliers
    outliers[col_name] = {'below': outliers_below, 'above': outliers_above}

# Print the results
for col_name, counts in outliers.items():
    print(f"Column: {col_name}")
    print(f"  Outliers below lower bound: {counts['below']}")
    print(f"  Outliers above upper bound: {counts['above']}\n")
    



Column: YearlyIncome
  Outliers below lower bound: 0
  Outliers above upper bound: 309

Column: TotalChildren
  Outliers below lower bound: 0
  Outliers above upper bound: 0

Column: NumberChildrenAtHome
  Outliers below lower bound: 0
  Outliers above upper bound: 3260



The 'YearlyIncome' and 'NumberChildrenAtHome' fields have a significant number of outliers above their respective upper bounds. This suggests that there are customers with unusually high yearly incomes and a number of children at home compared to the general population of the dataset. The absence of outliers for 'TotalChildren' suggests that the distribution of this variable is relatively tight, with most data points falling within a normal range.

**Handling Outliers through Capping in DimCustomer**

In [8]:

    # Cap outliers
dimCustomer = dimCustomer.withColumn(col_name, when(col(col_name) < lit(lower_bound), lit(lower_bound))
                                               .when(col(col_name) > lit(upper_bound), lit(upper_bound))
                                               .otherwise(col(col_name)))

# Update the DataFrame in dictionary
dataframes['DimCustomer'] = dimCustomer


**Counting Outliers again**

In [9]:
dimCustomer = dataframes['DimCustomer']

# List of numerical columns to check for outliers
numerical_columns = ['YearlyIncome', 'TotalChildren', 'NumberChildrenAtHome'] 

outliers = {}
for col_name in numerical_columns:
    # Calculate the lower and upper quartiles
    bounds = dimCustomer.stat.approxQuantile(col_name, [0.25, 0.75], 0.05)
    IQR = bounds[1] - bounds[0]

    # Define lower and upper bounds for outliers
    lower_bound = bounds[0] - 1.5 * IQR
    upper_bound = bounds[1] + 1.5 * IQR

    # Count outliers
    outliers_below = dimCustomer.filter(col(col_name) < lit(lower_bound)).count()
    outliers_above = dimCustomer.filter(col(col_name) > lit(upper_bound)).count()

    # Store the count of outliers
    outliers[col_name] = {'below': outliers_below, 'above': outliers_above}

# Print the results
for col_name, counts in outliers.items():
    print(f"Column: {col_name}")
    print(f"  Outliers below lower bound: {counts['below']}")
    print(f"  Outliers above upper bound: {counts['above']}\n")
    



Column: YearlyIncome
  Outliers below lower bound: 0
  Outliers above upper bound: 309

Column: TotalChildren
  Outliers below lower bound: 0
  Outliers above upper bound: 0

Column: NumberChildrenAtHome
  Outliers below lower bound: 0
  Outliers above upper bound: 0



The output shows that in the 'DimCustomer' dataset, 'YearlyIncome' has 309 outliers above the upper bound, while 'TotalChildren' and 'NumberChildrenAtHome' have no outliers either below the lower bound or above the upper bound.

**Checking Outliers in FactInternetSales**

In [10]:
factInternetSales = dataframes['FactInternetSales']

# List of numerical columns to check for outliers
numerical_columns1 = ['OrderQuantity', 'UnitPrice', 'ExtendedAmount', 'SalesAmount', 'TaxAmt', 'Freight'] 

outliers = {}
for col_name in numerical_columns1:
    # Calculate the lower and upper quartiles
    bounds = factInternetSales.stat.approxQuantile(col_name, [0.25, 0.75], 0.05)
    IQR = bounds[1] - bounds[0]

    # Define lower and upper bounds for outliers
    lower_bound = bounds[0] - 1.5 * IQR
    upper_bound = bounds[1] + 1.5 * IQR

    # Count outliers
    outliers_below = factInternetSales.filter(col(col_name) < lit(lower_bound)).count()
    outliers_above = factInternetSales.filter(col(col_name) > lit(upper_bound)).count()

    # Store the count of outliers
    outliers[col_name] = {'below': outliers_below, 'above': outliers_above}

# Print the results
for col_name, counts in outliers.items():
    print(f"Column: {col_name}")
    print(f"  Outliers below lower bound: {counts['below']}")
    print(f"  Outliers above upper bound: {counts['above']}\n")


Column: OrderQuantity
  Outliers below lower bound: 0
  Outliers above upper bound: 0

Column: UnitPrice
  Outliers below lower bound: 0
  Outliers above upper bound: 15454

Column: ExtendedAmount
  Outliers below lower bound: 0
  Outliers above upper bound: 15454

Column: SalesAmount
  Outliers below lower bound: 0
  Outliers above upper bound: 15454

Column: TaxAmt
  Outliers below lower bound: 0
  Outliers above upper bound: 15454

Column: Freight
  Outliers below lower bound: 0
  Outliers above upper bound: 15454



**Handling Outliers in FactInternetSales using Segmentation:**
    High values in FactInternetSales (like UnitPrice, SalesAmount, etc.) may represent large, legitimate transactions (e.g., bulk purchases, high-value items). Removing these outliers might lead to losing insights into a potentially significant customer segment. Segmenting sales data into different categories, such as regular sales and high-value sales allows for more nuanced analysis and understanding of different customer behaviors. I will perform segmentation later. 

**Exploratory Data Analysis (EDA) of DimCustomer**

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt # For plotting
import numpy as np
import seaborn as sns

dim_customer_df = dataframes['DimCustomer']
# Convert PySpark DataFrame to Pandas DataFrame for tabular view
pandas_dim_customer_df = dim_customer_df.toPandas()

# Display the first few rows in a tabular format
pandas_dim_customer_df.head()

In [ ]:
pandas_dim_customer_df.describe()

In [ ]:
pandas_dim_customer_df.info()

In [ ]:
# Selecting specific columns to create a new DataFrame
new_df = pandas_dim_customer_df[['YearlyIncome', 'TotalChildren', 'NumberChildrenAtHome']]
sns.set_theme(style="whitegrid")
plt.figure(figsize=(12, 16))
sns.pairplot(new_df)
plt.show()

**Here are the observations:**

**'YearlyIncome' Distribution:** The histogram for 'YearlyIncome' indicates a right-skewed distribution, suggesting that most customers have a lower income, with fewer customers earning higher incomes.

**'TotalChildren' Distribution:** The 'TotalChildren' histogram shows a relatively uniform distribution for lower counts of children, but it decreases as the number of children increases, indicating fewer customers have a larger number of children.

**'NumberChildrenAtHome' Distribution:** The histogram for 'NumberChildrenAtHome' indicates that a majority of customers have no children at home or a small number, with the frequency decreasing as the number of children at home increases.

**'YearlyIncome' vs. 'TotalChildren':** The scatter plot does not indicate a clear correlation between 'YearlyIncome' and 'TotalChildren'. There is a wide spread of incomes for customers with few children, and as the number of children increases, the range of incomes seems to narrow, but this could be due to fewer data points.

**'YearlyIncome' vs. 'NumberChildrenAtHome':** Similar to 'TotalChildren', there is no apparent correlation between 'YearlyIncome' and 'NumberChildrenAtHome'. Incomes are spread across the range for customers with few children at home.

**'TotalChildren' vs. 'NumberChildrenAtHome':** The scatter plot shows a pattern where as the 'TotalChildren' increases, 'NumberChildrenAtHome' also tends to increase, which is an expected relationship. However, there are many instances where customers have more 'TotalChildren' but none or fewer 'NumberChildrenAtHome', which may indicate children who have grown up and moved out.

In [ ]:
import pandas as pd

# File path
file_path = 'C:/Users/rasman khurshid/Desktop/Data Processing/Dataset_Final/DataSet_final/DimCustomer.csv'

# Try different encodings if 'utf-8' doesn't work
try:
    df = pd.read_csv(file_path, encoding='utf-8')
except UnicodeDecodeError:
    try:
        df = pd.read_csv(file_path, encoding='latin1')
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding='iso-8859-1')  # or try 'cp1252'
        
print(sorted(df['NumberChildrenAtHome'].unique()))

# Convert the 'NumberChildrenAtHome' from float to int
df['NumberChildrenAtHome'] = df['NumberChildrenAtHome'].astype(int)

# Now check the unique values again
print(sorted(df['NumberChildrenAtHome'].unique()))


# Set the aesthetic style of the plots
sns.set_theme(style="whitegrid")

# Increase the figure size to make the plot wider
plt.figure(figsize=(10, 12))  # Adjust the size as needed

# Create a bar plot for 'NumberChildrenAtHome' column
ax = sns.countplot(x='NumberChildrenAtHome', data=df)

# Set the title and labels of the plot
ax.set_title('Count of Number of Children at Home')
ax.set_xlabel('Number of Children at Home')
ax.set_ylabel('Count')

# Show the plot
plt.show()




## Distribution of Total Children Across Yearly Income

In [ ]:
plt.figure(figsize=(16, 8))
sns.violinplot(x='YearlyIncome', y='TotalChildren', data=pandas_dim_customer_df)
plt.title('Distribution of Total Children Across Yearly Income Ranges')
plt.xlabel('Yearly Income')
plt.ylabel('Total Children')
plt.show()

**Analysis:** The violin plot visualizes the distribution of 'Total Children' across various 'Yearly Income' ranges. Each violin shape corresponds to a different income bracket, showing the distribution of family sizes within that bracket. The plot suggests a relatively consistent median of around 1 to 2 children across all income ranges, as indicated by the white dot within each violin. The thickness of each violin indicates the density of data points at different counts of 'Total Children', with the thickest part representing the most common value. Notably, there are no significant outliers or extreme variations in family size across different income levels, as all violins maintain a similar shape and width throughout their length. The consistency across the income ranges suggests there is no strong correlation between the number of children and yearly income within this dataset. The range of 'Total Children' is mostly between 0 and 3 for all income levels, indicating that most families in this dataset have fewer than 3 children, regardless of income.


In [ ]:
plt.figure(figsize=(16, 8))
sns.boxplot(x='YearlyIncome', y='TotalChildren', data=pandas_dim_customer_df)
plt.title('Box Plot of Total Children Across Yearly Income Ranges')
plt.xlabel('Yearly Income')
plt.ylabel('Total Children')
plt.show()

**Age Distribution of Customers**

In [ ]:
from datetime import datetime

# Convert BirthDate from string to datetime
pandas_dim_customer_df['BirthDate'] = pd.to_datetime(pandas_dim_customer_df['BirthDate'])

# Calculate age
current_year = datetime.now().year
pandas_dim_customer_df['Age'] = current_year - pandas_dim_customer_df['BirthDate'].dt.year
# Convert BirthDate from string to datetime
pandas_dim_customer_df['BirthDate'] = pd.to_datetime(pandas_dim_customer_df['BirthDate'])

plt.figure(figsize=(10, 6))
sns.histplot(pandas_dim_customer_df['Age'], bins=20, kde=True)
plt.title('Age Distribution of Customers')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()


**Histogram Analysis:** 
The distribution of age appears to be right-skewed, meaning there are more younger customers than older ones.
The bulk of the customers seem to fall within the range of approximately 50 to 70 years old.
There is a long tail extending into the older age ranges, suggesting a smaller number of much older customers. The skew towards younger ages suggests a potentially significant segment of the market. Marketing strategies targeting this demographic could focus on products or services that appeal to a younger audience.

**Exploratory Data Analysis (EDA) for DimGeography**

In [ ]:
dim_geo_df = dataframes['DimGeography']
# Convert PySpark DataFrame to Pandas DataFrame for tabular view
pandas_dim_geo_df = dim_geo_df.toPandas()

# Display the first few rows in a tabular format
pandas_dim_geo_df.head()

In [ ]:
pandas_dim_geo_df.describe()

In [ ]:
pandas_dim_geo_df.info()


**Understanding how customers are distributed geographically**

In [ ]:

# Distribution by City
city_counts = pandas_dim_geo_df['City'].value_counts()
# Limit to top N cities for better readability
top_cities = city_counts.head(20)  # Adjust N as needed

plt.figure(figsize=(14, 8))  # Increase figure size
sns.barplot(y=top_cities.index, x=top_cities.values, palette='viridis')  # Horizontal bar chart
plt.title('Top 20 Cities by Number of Customers')
plt.xlabel('Number of Customers')
plt.ylabel('City')
plt.show()




In [ ]:
# Distribution by StateProvinceName
state_counts = pandas_dim_geo_df['StateProvinceName'].value_counts()
plt.figure(figsize=(12, 7))
sns.barplot(x=state_counts.index, y=state_counts.values, palette='viridis') 
plt.title('Distribution of Customers by State/Province')
plt.xlabel('State/Province')
plt.ylabel('Number of Customers')
plt.xticks(rotation=90)  
plt.show()



In [ ]:

# Distribution by CountryRegionCode
country_counts = pandas_dim_geo_df['CountryRegionCode'].value_counts()
plt.figure(figsize=(12, 7))
sns.barplot(x=country_counts.index, y=country_counts.values, palette='viridis') 
plt.title('Distribution of Customers by Country')
plt.xlabel('Country')
plt.ylabel('Number of Customers')
plt.xticks(rotation=45) 
plt.show()

**Analysis:** Together, these plots provide an understanding of where the customer base is concentrated. The US dominates the customer distribution, with significant counts in key European cities and Canadian provinces.

**Exploratory Data Analysis (EDA) for DimSalesTerritory**

In [ ]:
dim_saleTerritory_df = dataframes['DimSalesTerritory']
# Convert PySpark DataFrame to Pandas DataFrame for tabular view
pandas_dim_saleTerritory_df = dim_saleTerritory_df.toPandas()

# Display the first few rows in a tabular format
pandas_dim_saleTerritory_df.head()

In [ ]:
pandas_dim_saleTerritory_df.describe()

In [ ]:
pandas_dim_saleTerritory_df.info()


**Exploratory Data Analysis (EDA) for FactInternetSales**

In [ ]:
dim_factInternet_df = dataframes['FactInternetSales']
# Convert PySpark DataFrame to Pandas DataFrame for tabular view
pandas_dim_factInternet_df = dim_factInternet_df.toPandas()

# Display the first few rows in a tabular format
pandas_dim_factInternet_df.head()

In [ ]:
pandas_dim_factInternet_df.describe()

In [ ]:
pandas_dim_factInternet_df.info()

## Sales Performance by Sales Territoty Region

In [ ]:
# Merge the sales data with the sales territory data on SalesTerritoryKey
merged_df = pd.merge(pandas_dim_factInternet_df, pandas_dim_saleTerritory_df, on='SalesTerritoryKey', how='left')

# Aggregate sales by territory
sales_by_territory = merged_df.groupby('SalesTerritoryRegion')['SalesAmount'].sum().sort_values(ascending=False)

# Convert the aggregated data to a DataFrame for visualization
sales_by_territory_df = sales_by_territory.reset_index()

# Visualize sales performance by territory
plt.figure(figsize=(12, 7))
sns.barplot(x='SalesTerritoryRegion', y='SalesAmount', data=sales_by_territory_df)
plt.title('Sales Performance by Sales Territory Region')
plt.xlabel('Sales Territory Region')
plt.ylabel('Total Sales')
plt.xticks(rotation=45)
plt.show()




A DataFrame merged_df is created by merging sales data with sales territory data on the 'SalesTerritoryKey'.
The data is then grouped by 'SalesTerritoryRegion' and the sales amounts are summed.
The aggregated sales data is sorted in descending order and reset into a new DataFrame for visualization purposes.
A bar plot is created with Seaborn, showing 'SalesTerritoryRegion' on the x-axis and 'SalesAmount' on the y-axis.
The title, axis labels, and rotation of the x-axis tick labels are set to make the chart more readable.

In [ ]:
# Create a pie chart to visualize the sales performance by region
plt.figure(figsize=(8, 8))
plt.pie(sales_by_region, labels=sales_by_region.index, autopct='%1.1f%%', startangle=140)
plt.title('Sales Performance by Sales Territory Region')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

# Display the pie chart
plt.show()

These charts are useful for quickly identifying which regions are leading in sales and which are lagging, which can inform strategic business decisions such as where to allocate resources or where to implement changes to improve sales performance.

**Handling Outliers in FactInternetSales using Segmentation**

In [ ]:
from pyspark.sql.functions import when, col, lit, count

factInternetSales = dataframes['FactInternetSales']

# Determine the threshold for high-value sales, e.g., 75th percentile of SalesAmount
threshold = factInternetSales.approxQuantile('SalesAmount', [0.75], 0.05)[0]

# Create a new column for segment identification
factInternetSales = factInternetSales.withColumn('SaleType', 
                                                 when(col('SalesAmount') > lit(threshold), lit('High-Value'))
                                                 .otherwise(lit('Regular')))

# Update the DataFrame in dictionary
dataframes['FactInternetSales'] = factInternetSales
# Group by the 'SaleType' and count the number of occurrences in each group
segmentation_count = factInternetSales.groupBy('SaleType').agg(count('*').alias('Count'))

factInternetSalesPandas = factInternetSales.toPandas()
factInternetSalesPandas.head()
segmentation_count_pandas = segmentation_count.toPandas()
print(segmentation_count_pandas)


In [ ]:
# Bar Chart for Segment Counts
segmentation_count_pandas.plot(kind='bar', x='SaleType', y='Count')
plt.xlabel('Sale Type')
plt.ylabel('Count')
plt.title('Count of Sales by Type')
plt.show()


# Predicting High-Value Customers using Logistic Regression 

Objective: To identify and predict high-value customers based on their purchase history, demographics, and geographic information.

Rationale: This use case is important because it helps the business focus its marketing and customer service efforts on individuals who are likely to generate the most revenue. By identifying high-value customers, the company can optimize resource allocation and tailor its strategies to retain these key customers.

**Performing Joins** 

In [ ]:

customer_geo_joined = dataframes['DimCustomer'].join(dataframes['DimGeography'], 'GeographyKey', 'left')

sales_customer_joined = dataframes['FactInternetSales'].join(customer_geo_joined, 'CustomerKey', 'inner')
final_joined_df = sales_customer_joined.join(dataframes['DimSalesTerritory'], 'SalesTerritoryKey', 'left')
# Join DimCustomer with DimGeography
customer_geo_joined = dataframes['DimCustomer'].join(dataframes['DimGeography'], 'GeographyKey', 'left')

# Join FactInternetSales with the combined DimCustomer and DimGeography
sales_customer_joined = dataframes['FactInternetSales'].join(customer_geo_joined, 'CustomerKey', 'inner')

# Join the result with DimSalesTerritory
final_joined_df = sales_customer_joined.join(dataframes['DimSalesTerritory'], 'SalesTerritoryKey', 'left')
from pyspark.sql import functions as F

# Calculate the average sales amount by sales territory
avg_sales_by_territory = factInternetSales.groupBy('SalesTerritoryKey').agg(
    F.avg('SalesAmount').alias('AvgSalesAmount'),
    F.sum(F.when(col('SaleType') == 'High-Value', 1).otherwise(0)).alias('HighValueSalesCount'),
    F.count('*').alias('TotalSalesCount')
)

# Calculate the proportion of high-value sales
avg_sales_by_territory = avg_sales_by_territory.withColumn(
    'HighValueSalesProportion', 
    col('HighValueSalesCount') / col('TotalSalesCount')
)

# Join this data back to the main DataFrame
final_joined_df = final_joined_df.join(avg_sales_by_territory, 'SalesTerritoryKey', 'left')
# Drop the duplicate column after the join
# Drop one of the duplicate 'SalesTerritoryKey' columns
final_joined_df = final_joined_df.drop("SalesTerritoryKey")

final_joined_df.printSchema()


**Data Cleaning** 

In [ ]:
from pyspark.sql.functions import col

# Check for missing values in each column
missing_values = final_joined_df.select([count(when(col(c).isNull(), c)).alias(c) for c in final_joined_df.columns]).collect()
print(missing_values)

final_joined_df = final_joined_df.dropDuplicates()

final_joined_df = final_joined_df.withColumn("NumberChildrenAtHome", col("NumberChildrenAtHome").cast("int"))
# Print the data types of each column
final_joined_df.dtypes


**Checking for any possible data leakage** 

In [ ]:
from pyspark.sql.functions import corr

final_joined_df = final_joined_df.withColumn("SaleTypeNumeric", when(col("SaleType") == "High-Value", 1).otherwise(0))

# List of numerical features to check for correlation
numerical_features = ['YearlyIncome', 'TotalChildren', 'NumberChildrenAtHome', 'OrderQuantity', 'UnitPrice'] 
# Calculate correlations and collect them into a dictionary
correlations = {}
for feature in numerical_features:
    correlation = final_joined_df.stat.corr("SaleTypeNumeric", feature)
    correlations[feature] = correlation

# Print out correlations
for feature, correlation in correlations.items():
    print(f"Correlation between SaleType and {feature}: {correlation}")


**Logistic Regression**

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator  # Importing the evaluator


# Define categorical and numerical features
categorical_cols = ['MaritalStatus', 'Gender', 'EnglishEducation', 'City', 'StateProvinceName', 'CountryRegionCode', 'SalesTerritoryRegion']
numerical_cols = ['YearlyIncome', 'TotalChildren', 'NumberChildrenAtHome', 'OrderQuantity', 'UnitPrice']

# StringIndexer for categorical columns (including target variable 'SaleType')
indexers = [StringIndexer(inputCol=c, outputCol=c+"_indexed").fit(final_joined_df) for c in categorical_cols]
indexers.append(StringIndexer(inputCol='SaleType', outputCol='label').fit(final_joined_df))

# OneHotEncoder for indexed categorical columns
encoders = [OneHotEncoder(inputCols=[c+"_indexed"], outputCols=[c+"_encoded"]) for c in categorical_cols]

# VectorAssembler to create feature vector
assemblerInputs = [c+"_encoded" for c in categorical_cols] + numerical_cols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")

# Initialize Logistic Regression model
lr = LogisticRegression(featuresCol="features", labelCol="label")

# Set up the Pipeline
pipeline = Pipeline(stages=indexers + encoders + [assembler, lr])

# Split the data into training and test sets
(train, test) = final_joined_df.randomSplit([0.7, 0.3])

# Train the model
model = pipeline.fit(train)

# Make predictions
predictions = model.transform(test)

# Evaluate the model
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate the model using Area Under ROC
evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
roc_auc = evaluator.evaluate(predictions)
print(f"Area Under ROC: {roc_auc}")

# Optionally, I can also evaluate using Area Under Precision-Recall Curve
evaluator.setMetricName("areaUnderPR")
pr_auc = evaluator.evaluate(predictions)
print(f"Area Under Precision-Recall Curve: {pr_auc}")


## Confusion Matrix

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Convert the PySpark DataFrame to a Pandas DataFrame
predictions_pd = predictions.select("label", "prediction").toPandas()

# Compute the confusion matrix
conf_matrix = confusion_matrix(predictions_pd['label'], predictions_pd['prediction'])

# Calculate metrics
accuracy = accuracy_score(predictions_pd['label'], predictions_pd['prediction'])
precision = precision_score(predictions_pd['label'], predictions_pd['prediction'])
recall = recall_score(predictions_pd['label'], predictions_pd['prediction'])
f1 = f1_score(predictions_pd['label'], predictions_pd['prediction'])

# Print Confusion Matrix and Metrics
print("Confusion Matrix:")
print(conf_matrix)

print("\nPerformance Metrics:")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


**Confusion Matrix:**

True Negatives (TN): 12775 (cases correctly predicted as negative)
False Positives (FP): 1 (cases incorrectly predicted as positive)
False Negatives (FN): 23 (cases incorrectly predicted as negative)
True Positives (TP): 5132 (cases correctly predicted as positive)
Performance Metrics:

Accuracy: Approximately 99.87%, indicating that the model correctly predicted the label for nearly all of the test data.
Precision: Approximately 99.98%, showing that almost all of the instances predicted as positive are indeed positive.
Recall: Approximately 99.55%, indicating that the model successfully identified most of the positive cases.
F1 Score: Approximately 99.77%, which is the harmonic mean of precision and recall, also indicating a very high performance.
These results suggest that the model is highly effective at classifying the test data with both high precision and high recall, which is often a challenging balance to achieve in practice. The model successfully identifies positive cases while maintaining a low rate of false positives.

**Plotting Logistic Regression**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


predictions_pd = predictions.toPandas()

# Set the size of the plot for better visibility
plt.figure(figsize=(14, 6))

# Create a subplot for High-Value probabilities
plt.subplot(1, 2, 1)
plt.hist(predictions_pd['probability'].apply(lambda x: x[1]), bins=20, alpha=0.5, color='blue')
plt.title('Probability Distribution for High-Value')
plt.xlabel('Probability')
plt.ylabel('Frequency')

# Create a subplot for Regular probabilities
plt.subplot(1, 2, 2)
plt.hist(predictions_pd['probability'].apply(lambda x: x[0]), bins=20, alpha=0.5, color='red')
plt.title('Probability Distribution for Regular')
plt.xlabel('Probability')
plt.ylabel('Frequency')

# Show the plot with a tight layout to ensure no overlap of subplots
plt.tight_layout()
plt.show()



**From the histograms, we can see that:**

The 'High-Value' probabilities (on the left) have a concentration of frequencies at the lower probability values and another concentration at the higher probability values. This bimodal distribution suggests that the model is quite certain about a portion of the 'High-Value' predictions, while another portion is predicted with low probability.
The 'Regular' probabilities (on the right) are predominantly concentrated at the higher probability values. This indicates that the model is very confident in predicting regular sales, with most of the predicted probabilities close to 1.
These histograms provide a much clearer separation of the two predicted classes and can help in understanding the model's confidence in its predictions. It appears that the model is very confident in predicting 'Regular' sales, while it has a spread of confidence levels for 'High-Value' sales. This could be indicative of the model having a better grasp on the characteristics that define 'Regular' sales compared to 'High-Value' sales.

# Predicting Number of Car Owned Using Linear Regression 

**Objective:** To predict Number of Cars Owned by customers based on their yearly income and demographics 

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# Load the dataset
file_path = 'C:/Users/rasman khurshid/Desktop/Data Processing/Dataset_Final/DataSet_final/DimCustomer.csv'
dim_customer_df = pd.read_csv(file_path, encoding='ISO-8859-1')

# Encoding 'CommuteDistance' as an ordinal variable based on its unique values
commute_distance_mapping = {
    '0-1 Miles': 1,
    '1-2 Miles': 2,
    '2-5 Miles': 3,
    '5-10 Miles': 4,
    '10+ Miles': 5
}
dim_customer_df['CommuteDistanceEncoded'] = dim_customer_df['CommuteDistance'].map(commute_distance_mapping)


In [ ]:
# EDA: Correlation Matrix and Heatmap
# Relevant columns for EDA
eda_columns = ['YearlyIncome', 'HouseOwnerFlag', 'TotalChildren', 
               'NumberChildrenAtHome', 'CommuteDistanceEncoded', 'NumberCarsOwned']
eda_df = dim_customer_df[eda_columns]
correlation_matrix = eda_df.corr()
#plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Correlation Matrix of Selected Features")
plt.show()

In [ ]:
# Printing the correlation matrix in a table format
correlation_matrix

**Analysis**: The correlation matrix and heatmap provide a visual and numerical representation of the relationship between various features and the target variable 'NumberCarsOwned'. The heatmap uses color intensity to indicate the strength and direction of the correlation, where red tones represent positive correlations and blue tones represent negative correlations.

From the analysis, 'YearlyIncome' shows the strongest positive correlation (0.47) with the target variable, suggesting that as income increases, the number of cars owned tends to increase as well. 'NumberChildrenAtHome' also has a strong positive correlation (0.42) with 'NumberCarsOwned', which implies that households with more children at home may require more cars. 'CommuteDistanceEncoded' has a moderate positive correlation (0.42) with the target, indicating that longer commute distances might be associated with owning more cars. On the other hand, 'HouseOwnerFlag' has a negligible negative correlation (-0.06) with 'NumberCarsOwned', hinting that home ownership status might not be a significant predictor of car ownership in this dataset.

The correlation between 'YearlyIncome' and 'NumberCarsOwned' is particularly noteworthy as it suggests a potential avenue for predictive modeling. While correlation does not imply causation, these relationships provide a useful starting point for building regression models to predict car ownership. It's important to consider these insights in the context of the domain when selecting features for modeling and interpreting the results of any predictive analysis.

**Linear Regression**

In [ ]:
# Proceeding with model training
# Preprocessing
relevant_columns = ['YearlyIncome', 'HouseOwnerFlag', 'TotalChildren', 
                    'NumberChildrenAtHome', 'CommuteDistanceEncoded', 'NumberCarsOwned']

# Note: Make sure 'CommuteDistanceEncoded' is treated as numerical if it's ordinal
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['YearlyIncome', 'TotalChildren', 'NumberChildrenAtHome', 'CommuteDistanceEncoded'])
    ])

# Separating features and target variable
X = dim_customer_df[relevant_columns[:-1]]  # Exclude 'NumberCarsOwned' from features
y = dim_customer_df['NumberCarsOwned']

# Applying transformations
X_transformed = preprocessor.fit_transform(X)

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.3, random_state=42)

# Model Training
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# Feature Importance Analysis
# Get the feature names after transformation
feature_names = preprocessor.named_transformers_['num'].get_feature_names_out()

# Combine feature names and coefficients from the model
feature_importances = pd.DataFrame(zip(feature_names, lr_model.coef_), 
                                   columns=['Feature', 'Coefficient'])

# Sort the features by the absolute magnitude of their coefficients
feature_importances['Absolute Coefficient'] = feature_importances['Coefficient'].abs()
feature_importances = feature_importances.sort_values(by='Absolute Coefficient', ascending=False)

# Predicting on the test set
y_pred = lr_model.predict(X_test)

# Model Evaluation
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

# Output the performance metrics
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R²):", r2)




Here's a step-by-step explanation of the code and its components:

**Preprocessing:**

relevant_columns are defined, which include both features and the target variable 'NumberCarsOwned'.
A ColumnTransformer with StandardScaler is set up to standardize the numerical features, which include 'YearlyIncome', 'TotalChildren', 'NumberChildrenAtHome', and 'CommuteDistanceEncoded'. Standardization is a common preprocessing step that scales the data so that it has a mean of 0 and a standard deviation of 1, helping to ensure that the scale of the features does not unduly influence the model.
Feature-Target Split:

The features (X) and the target variable (y) are separated. X includes all the relevant columns except for the target variable 'NumberCarsOwned'.
Data Transformation:

The features are transformed using the preprocessor, which standardizes the numerical features.
Train-Test Split:

The dataset is split into training and testing sets using train_test_split. This allows the model to be trained on one portion of the data and then evaluated on a separate portion to test its performance on unseen data. The test size is set to 30% of the data, with the random state fixed at 42 for reproducibility.
Model Training:

A LinearRegression model is instantiated and fitted to the training data. This model learns the relationship between the features and the target variable.
Feature Importance Analysis:

After training, the feature names are retrieved and combined with the model's coefficients, which represent the influence of each standardized feature on the target variable. The absolute values of these coefficients are calculated to assess the magnitude of the feature's impact, regardless of the direction (positive or negative).
Prediction and Evaluation:

The model is used to predict the target variable for the test set.
The model's performance is evaluated using two metrics: Root Mean Squared Error (RMSE) and R-squared (R²). RMSE measures the average magnitude of the errors between the predicted and actual values; a lower RMSE indicates better fit. R² measures the proportion of the variance in the target variable that can be explained by the model's features; an R² of 1 indicates perfect prediction, while an R² of 0 indicates that the model explains none of the variance.

**Analysis of Output:**

RMSE (0.8984): This value indicates the standard deviation of the prediction errors. In this context, it means that the model's predictions are, on average, 0.8984 units away from the actual number of cars owned. Whether this is acceptable depends on the context and the range of 'NumberCarsOwned' in the dataset.

R² (0.3765): This value suggests that approximately 37.65% of the variance in 'NumberCarsOwned' can be explained by the model. This is a moderate amount of variance explained, indicating the model has some predictive power, but there is still a significant amount of variance that the model does not account for.

The analysis implies that while the model has learned some of the underlying patterns in the data, there's room for improvement. Potential steps to improve the model could include feature engineering, considering non-linear models, or addressing potential outliers and leverage points.

## Confusion Matrix

In [ ]:
y_pred_rounded = np.rint(y_pred)  # Round predictions to the nearest integer

# Compute the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_rounded)

# Output the confusion matrix
print("Confusion Matrix:")
print(conf_matrix)


**Class-wise Breakdown:**

Class 0 (First Row): 55 correct predictions, but 984 instances were incorrectly predicted as Class 1, 167 as Class 2, 7 as Class 3.
Class 1 (Second Row): 1029 correct predictions, but 21 instances were incorrectly predicted as Class 0, 406 as Class 2, 22 as Class 3.
Class 2 (Third Row): 799 correct predictions, with significant misclassifications into Classes 1 and 3.
Class 3 and Class 4 (Fourth and Fifth Rows): These classes have a higher number of misclassifications compared to correct predictions.
Implications:

The model seems to struggle most with accurately predicting Class 0, often confusing it with Class 1.
There is a considerable amount of confusion between Classes 1 and 2, and Classes 3 and 4.
The higher classes (like 3 and 4) have relatively few correct predictions, which might be due to a lower number of instances of these classes in the dataset (class imbalance).

**Possible Actions:**

Improve Class Imbalance: If there's a class imbalance in the dataset, techniques like oversampling the minority class, undersampling the majority class, or using class weights can help.
Feature Engineering: Look into creating or transforming features that might help the model distinguish between classes better.
Model Complexity: Consider whether a more complex model or different algorithms might capture the nuances between classes better.
Hyperparameter Tuning: Experiment with different hyperparameters to see if the model's performance can be improved.

## Feature Importance Analysis:
After training the model, checking the feature importances or coefficients to see which features are having the most impact on predictions.

In [ ]:
# Displaying feature importances
feature_importances

**Analyzing the feature importances:**

CommuteDistanceEncoded (Coefficient: 0.344798): This feature has the highest coefficient in absolute terms, which indicates that among the features used, 'CommuteDistanceEncoded' has the strongest relationship with the number of cars owned. Since this is a positive coefficient, a longer commute distance is associated with owning more cars.

YearlyIncome (Coefficient: 0.292740): 'YearlyIncome' is also a strong predictor of the number of cars owned, with the second-highest coefficient. The positive sign means that as yearly income increases, the number of cars owned tends to increase.

NumberChildrenAtHome (Coefficient: 0.272344): This feature has the third-highest impact on the number of cars owned. The positive coefficient suggests that households with more children at home are likely to own more cars.

TotalChildren (Coefficient: 0.063826): 'TotalChildren' has the smallest positive impact on the number of cars owned among the features listed. This indicates a weaker relationship compared to the other features.
'CommuteDistanceEncoded', 'YearlyIncome', and 'NumberChildrenAtHome' are the most significant predictors of car ownership in the model. The magnitude of these coefficients suggests that they are important factors to consider when predicting the number of cars a person or household owns.

In [ ]:
# Actual vs. Predicted plot
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y.min(), y.max()], [y.min(), y.max()], '--k')  # Diagonal line
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs. Predicted')
plt.show()



**Actual vs. Predicted Plot:**

This plot displays the actual values of the target variable on the x-axis and the predicted values on the y-axis.
Ideal predictions would fall on the dashed diagonal line where the actual and predicted values are equal.
The plot shows that for most actual values, there are clusters of predicted values. This indicates that for any given actual value of 'NumberCarsOwned', there is a range of predictions, which suggests some inaccuracy in the model.
The clustering also suggests that the model may have difficulty distinguishing between different levels of the target variable, especially since the data points for actual values of 1, 2, and 3 cars seem to have overlapping predicted values.

In [ ]:
# Residuals plot
residuals = y_test - y_pred
plt.figure(figsize=(10, 6))
plt.scatter(y_pred, residuals, alpha=0.5)
plt.hlines(y=0, xmin=y_pred.min(), xmax=y_pred.max(), colors='red', linestyles='--')
plt.xlabel('Predicted')
plt.ylabel('Residuals')
plt.title('Residuals Plot')
plt.show()

**Residuals Plot:**

The residuals plot shows the residuals (the differences between the actual and predicted values) on the y-axis against the predicted values on the x-axis.
Ideally, residuals should be randomly scattered around the horizontal line at zero, indicating that the model's predictions are consistent across all values.
However, in this plot, we observe a clear pattern where the residuals do not scatter randomly but instead form distinct bands. This indicates systematic errors in the model's predictions.
The bands also suggest that the relationship between the predictors and the target may not be linear or that there may be other variables affecting the outcome that are not included in the model.

**Thats the End. Thank you** 